### Mine useful terms from different clusters
### Datasets are downloadable from  https://mars.cyverse.org/data_dumps/GEOME.txt.zip

#### replace GEOME with other relevant collection name to get the data set. 

#### Download, unzip, and put the .txt file in the appropriate folder according to the jupyter file


#### Author: Hong Cui



### Obtain clusters and their informative terms 

In [1]:
import sys
import re
import string
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import fasttext # pip install fasttext-0.9.2-cp310-cp310-win_amd64.whl
import pickle
from joblib import Parallel, delayed
from sklearn.cluster import Birch
import fastcluster
from scipy.cluster.hierarchy import fcluster
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 4000

import multiprocessing

cores = multiprocessing.cpu_count()
cores = min(1, cores-1)

In [2]:
# srcs = {
#         "GEOME": ['record_colloquialName', 'record_scientificName','record_kingdom', 'record_phylum',
#                   'record_subPhylum', 'record_order', 'record_infraOrder', 'record_superOrder',
#                   'record_subOrder', 'record_class', 'record_subClass', 'record_infraClass', 'record_tribe', 
#                   'record_subTribe', 'record_superFamily', 'record_family', 'record_subFamily', 'record_genus',
#                   'record_subGenus', 'record_specificEpithet', 'record_infraspecificEpithet', 'record_taxonRemarks', 
#                   'record_taxonRank', 'record_sex', 'record_preservative', 'record_fixative', 'record_relaxant', 
#                   'parent_fieldNotes','parent_locality', 'parent_verbatimLocality', 'parent_habitat', 
#                   'parent_microHabitat', 'record_morphospeciesDescription', 'record_lifeStage'], 
#         "OPENCONTEXT":['Temporal Coverage_label', 
#              'Has taxonomic identifier_label','Has anatomical identification_label',
#              'Consists of_label',  'Has type_label', 'item category', 'context label'], 
#         "SMITHSONIAN":[ 'scientificName', 'higherClassification', 'kingdom', 'phylum', 'class', 'order', 
#                        'family', 'genus', 'subgenus', 'specificEpithet', 'infraspecificEpithet', 'sex',
#                        'lifeStage', 'habitat', 'preparations', 'waterBody', 'occurrenceRemarks','locality'], 
#         "SESAR":['description_supplementMetadata_geologicalAge','description_collectionMethod', 
#                  'description_material',  'description_sampleType', 
#                  'description_supplementMetadata_classificationComment', 'description_description', 
#                  'description_supplementMetadata_purpose', 'description_collectionMethodDescr',
#                  'description_supplementMetadata_primaryLocationType', 
#                  'description_supplementMetadata_geologicalUnit', 'description_supplementMetadata_locality',
#                  'description_supplementMetadata_localityDescription', 'description_supplementMetadata_fieldName']
#        }

# srcs = {
#         "GEOME": ['record_scientificName', 'parent_locality'], 
#         "OPENCONTEXT":['Has taxonomic identifier_label','Has anatomical identification_label',
#              'Consists of_label',  'Has type_label', 'item category'], 
#         "SMITHSONIAN":[ 'scientificName', 'higherClassification', 'waterBody', 'locality'], 
#         "SESAR":['description_material',  'description_sampleType', 
#                  'description_supplementMetadata_purpose', 
#                  'description_supplementMetadata_primaryLocationType', 
#                  'description_supplementMetadata_geologicalUnit', 'description_supplementMetadata_locality',
#                  'description_supplementMetadata_fieldName']
#        }

srcs = {
        "GEOME": ['parent_habitat', 'parent_microHabitat'], 
#         "OPENCONTEXT": ['Consists of_label']     
#"OPENCONTEXT":['Consists of_label',  'Has type_label'] 
         "SMITHSONIAN":[ 'habitat'], 
#         "SESAR":['description_material',  'description_sampleType', 
#                  'description_supplementMetadata_purpose', 
#                  'description_supplementMetadata_primaryLocationType', 
#                  'description_supplementMetadata_fieldName']
       }

# srcs = {
#         "GEOME": ['record_scientificName', 'parent_locality'], 
#         "OPENCONTEXT":['Has taxonomic identifier_label','Has anatomical identification_label',
#              'Consists of_label',  'Has type_label', 'item category'], 
#         "SMITHSONIAN":[ 'scientificName', 'higherClassification', 'waterBody', 'locality'], 
#         "SESAR":['description_material',  'description_sampleType', 
#                  'description_supplementMetadata_purpose', 
#                  'description_supplementMetadata_primaryLocationType', 
#                  'description_supplementMetadata_fieldName']
#       }

srcids = {
        "GEOME": 'record_bcid', 
        #"OPENCONTEXT": 'citation uri', 
        "SMITHSONIAN":'occurrenceID', 
        #"SESAR":'igsn'
       }

#srcnames = ["GEOME", "OPENCONTEXT", "SMITHSONIAN", "SESAR"]

srcnames = ["GEOME", "SMITHSONIAN"]
       

### frequency counts of the entries in one collection and one field

In [3]:
def saveCounts2CSV (c, cname, threshold):
    th = pd.DataFrame(columns=[cname, "count"])
    for i in c.index:
        if c[i] >= threshold:
            th  =  th.append({cname:i,  "count":c[i]}, ignore_index = True)
    th.to_csv("counts "+ cname+".csv")

In [4]:
src="OPENCONTEXT"
data = pd.read_csv('data/'+src+'.txt',sep='#', keep_default_na=False, encoding='utf-8')


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (6,13,20,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
a = data['context label'].value_counts()
# if contains greater than 4000 rows, so it cann't be displayed fully by default
# print out entries in f one by one
for r in a.index:
    print(str(a[r]) +" :"+r)  #count: phrases
   
saveCounts2CSV(a, "OC context", 10)
    

b = data['Consists of_label'].value_counts()
b
saveCounts2CSV(b, "OC consists", 10)

c = data['Has type_label'].value_counts()
c
saveCounts2CSV(c, "OC type", 10)


d = data['item category'].value_counts()
d
saveCounts2CSV(d, "OC item category", 0)

4819 :Iran/Chogha Mish/Protoliterate
4591 :Iran/Chogha Mish/Middle Susiana
4224 :Ecuador/Galápagos/Hacienda El Progreso/Carpintero
3630 :Turkey/Çatalhöyük/Mound East/Area 4040/Unit 10396
3439 :Turkey/Çatalhöyük/Mound East/Area 4040/Unit 8864
3105 :Turkey/Çatalhöyük/Mound East/Area South/Unit 16262
2952 :Turkey/Çatalhöyük/Mound East/Area South/Unit 4121
2764 :Turkey/Çatalhöyük/Mound East/Area South/Unit 5290
2750 :Turkey/Çatalhöyük/Mound East/Area 4040/Unit 8859
2569 :Turkey/Barcın Höyük/Trench L13/Phase VID
2557 :Germany/Emden/Kirchstraße/Phase 2
2338 :Germany/Emden/Kirchstraße/Phase 3
2305 :Italy/Forcello Bagnolo San Vito/Sector R18/Context 876
2254 :Turkey/Çatalhöyük/Mound East/Area South/Unit 16253
2250 :Turkey/Ulucak/Level VIa
2179 :Turkey/Ulucak/Level IVb
2173 :Cyprus/Polis Chrysochous/B.D7:R14
2089 :Cyprus/Polis Chrysochous/B.D7:Q17
1857 :Cyprus/Polis Chrysochous/B.D7:n17
1740 :Turkey/Çatalhöyük/Mound East/Area South/Unit 15743
1656 :United States/Washington/Clallam County/Čḯxwic

44 :Turkey/Çatalhöyük/Mound East/Area Bach/Unit 8561
44 :Turkey/Erbaba Höyük/Area F/Strat Group H/Unit [Not indicated]
44 :Turkey/Çatalhöyük/Mound West/Trench 1/Unit 7787
44 :Turkey/Avkat Survey Area/SU Group 19/S1340
44 :Italy/Poggio Civitate/Tesoro/Tesoro 1AN/1968, ID:40
44 :Jordan/Petra Great Temple/Upper Temenos/Trench 105-106/Locus 8
44 :Turkey/Ilıpınar/Area 1/Lot 96
44 :Italy/Forcello Bagnolo San Vito/Sector [Blank]/Context UNKNOWN
44 :Turkey/Avkat Survey Area/SU Group 20/S1350
44 :Turkey/Domuztepe/I/Lot 2678
44 :Turkey/Domuztepe/I/Lot 3390
44 :United States/Florida/Volusia County/VO00024/B
44 :Turkey/Avkat Survey Area/SU Group 6/S1500
44 :Turkey/Avkat Survey Area/SU Group 5/S1764
44 :Italy/Forcello Bagnolo San Vito/Sector R19/Context 2355
44 :Egypt/Giza/Heit el-Ghurab/ETH/Feat. 20274
44 :Cyprus/PKAP Survey Area/Vigla/EU-5/SU-5408
44 :Jordan/Petra Great Temple/Upper Temenos/Trench 41/Locus 2/Seq. 41023
44 :Jordan/Petra Great Temple/Upper Temenos/Trench 105-106/Locus 1
44 :Turkey/

24 :Peru/Unnamed (Peru)/Unnamed Site or Source (Peru)
24 :Cyprus/PKAP Survey Area/Vigla/EU-14/SU-5105
24 :Italy/Poggio Civitate/Civitate A/Civitate A83/2013, ID:671/Locus 8
24 :Cyprus/PKAP Survey Area/Koutsopetria/EU-Γ9/SU-4026
24 :Turkey/Avkat Survey Area/SU Group 8/S1489
24 :Egypt/Giza/Heit el-Ghurab/WCE/Feat. 6329
24 :Cyprus/PKAP Survey Area/Vigla/EU-2/SU-5215
24 :Turkey/Çatalhöyük/Mound East/Area TP/Unit 6842
24 :Denmark/Ulkestrup Lyng
24 :Turkey/Kenan Tepe/Area E/Trench 2/Locus 25/Finds Bag 16
24 :Turkey/Kenan Tepe/Area A/Trench 1/Locus 1015
23 :Turkey/Avkat Survey Area/SU Group 11/S1911
23 :Turkey/Çatalhöyük/Mound East/Area Bach/Unit 8380
23 :Greece/Corinthia/EKAS Region/Xylokeriza/DU 34
23 :Turkey/Avkat Survey Area/SU Group 5/S1787
23 :Egypt/Giza/Heit el-Ghurab/AA/Feat. 602
23 :Turkey/Çatalhöyük/Mound East/Area 4040/Unit 16400
23 :Greece/Corinthia/EKAS Region/Yiriza/DU 1642
23 :Jordan/Petra Great Temple/Temple/Trench 73/Locus 34/Seq. 73436
23 :Turkey/Kenan Tepe/Area F/Trench 1/L

14 :Turkey/Kenan Tepe/Area D/Trench 4/Locus 4032/Finds Bag 4171
14 :Jordan/Petra Great Temple/Upper Temenos/Trench 77/Locus 21/Seq. 77123
14 :Egypt/Giza/Heit el-Ghurab/BBE/Feat. 6984
14 :Italy/Poggio Civitate/Tesoro/Tesoro 99/T99 2019/Locus 1
14 :Egypt/Giza/Heit el-Ghurab/WCE/Feat. 6340
14 :United States/Minnesota/Lake Carlos Beach Site/Context: Horiz. 17 / Vert. 20 to 25cm
14 :Jordan/Petra Great Temple/Temple/Trench 15/Locus 20/Seq. 15025B
14 :Turkey/Kenan Tepe/Area A/Trench 2/Locus 2012/Finds Bag 2123
14 :Turkey/Kenan Tepe/Area F/Trench 7/Locus 7113
14 :Jordan/Petra Great Temple/Propylaeum/Trench 99/Locus 1/Seq. 99002
14 :Ecuador/Buen Suceso/Trench 1/Unit N1-2 O1/Phase VII/Layer Valdivia
14 :Turkey/Çatalhöyük/Mound East/Area Bach/Unit 6119
14 :Jordan/Petra Great Temple/Lower Temenos/Trench 17/Locus 18/Seq. 17122
14 :Jordan/Khirbat al-Mudayna al-Aliya/Unit 3G46/Locus 7
14 :Ecuador/Buen Suceso/Trench 1/Unit N23-24 O1/Phase VII/Layer Post-V
14 :Turkey/Kenan Tepe/Area B/Trench 4/Locus 40

9 :Jordan/Petra Great Temple/Temple/Trench 85/Locus 1/Seq. 85103
9 :Italy/Forcello Bagnolo San Vito/Sector S18/Context 1687
9 :East Jerusalem/City of David, Area G/Context not specified
9 :Turkey/Kenan Tepe/Area G/Trench 9/Locus 18
9 :Turkey/Ilıpınar/Area 6/Lot 152
9 :Egypt/Giza/Heit el-Ghurab/WCE/Feat. 6448
9 :Turkey/Çatalhöyük/Mound East/Area North/Unit 1168
9 :Turkey/Avkat Survey Area/SU Group 13/S1425
9 :Jordan/Numayra/Unit NE 4-1/Locus 30
9 :Turkey/Kenan Tepe/Area D/Trench 6/Locus 170/Finds Bag 5
9 :Turkey/Çatalhöyük/Mound East/Area North/Unit 1157
9 :Jordan/Petra Great Temple/Upper Temenos/Trench 105-106/Locus 8/Seq. 105192
9 :Turkey/Kenan Tepe/Area C/Trench 1/Locus 1021
9 :Turkey/Kenan Tepe/Area E/Trench 2/Locus 31
9 :Cyprus/PKAP Survey Area/Kokkinokremos Plain/Unit 191
9 :Turkey/Domuztepe/I/Lot 611
9 :Israel/Hazor/Area A4/Locus 8336/Locus 8382/Basket 53128
9 :Turkey/Kenan Tepe/Area G/Trench 7/Locus 32/Finds Bag 2
9 :Cyprus/PKAP Survey Area/Koutsopetria/EU-12/SU-7029
9 :Turkey/D

...

1 :Jordan/Petra Great Temple/Lower Temenos/Trench 71/Locus 18/Seq. 71389
1 :Jordan/Petra Great Temple/Upper Temenos/Trench 105-106/Locus 1/Seq. 105949
1 :Jordan/Petra Great Temple/Upper Temenos/Trench 84/Locus 8/Seq. 84042
1 :United States/Minnesota/Lake Carlos Beach Site/Context: Horiz. 27 / Vert. 35 to 40cm
1 :Jordan/Petra Great Temple/Lower Temenos/Special Project 124/Locus 2/Seq. SP124052
1 :Jordan/Petra Great Temple/Lower Temenos/Trench 16/Locus 9/Seq. 16105D
1 :Jordan/Petra Great Temple/Lower Temenos/Trench 20/Locus 25/Seq. 20213K
1 :Jordan/Petra Great Temple/Temple/Trench 57/Locus 6/Seq. 57067
1 :Jordan/Petra Great Temple/Propylaeum/Trench 93/Locus 1/Seq. 93317
1 :Jordan/Petra Great Temple/Temple/Trench 57/Locus 12/Seq. 57395
1 :Jordan/Petra Great Temple/Lower Temenos/Trench 5/Locus 3/Seq. 5044
1 :Jordan/Petra Great Temple/Lower Temenos/Trench 71/Locus 28/Seq. 71488
1 :Jordan/Petra Great Temple/Temple/Trench 63/Locus 1/Seq. 63023
1 :Jordan/Petra Great Temple/Temple/Trench 55/Loc

1 :Turkey/Kenan Tepe/Area G/Trench 9/Locus 8/Finds Bag 2
1 :Turkey/Kenan Tepe/Area D/Trench 4/Locus 4072/Finds Bag 4
1 :Turkey/Kenan Tepe/Area D/Trench 4/Locus 4066/Finds Bag 4
1 :Turkey/Kenan Tepe/Area C/Trench 1/Locus 1043/Finds Bag 1411
1 :Turkey/Kenan Tepe/Area D/Trench 6/Locus 8/Finds Bag 15
1 :Turkey/Kenan Tepe/Area D/Trench 6/Locus 4/Finds Bag 73
1 :Turkey/Kenan Tepe/Area D/Trench 6/Locus 72/Finds Bag 12
1 :Turkey/Kenan Tepe/Area C/Trench 1/Locus 1034/Finds Bag 1264
1 :Turkey/Kenan Tepe/Area C/Trench 1/Locus 1039/Finds Bag 1216
1 :Turkey/Kenan Tepe/Area D/Trench 6/Locus 75/Finds Bag 3
1 :Turkey/Kenan Tepe/Area D/Trench 6/Locus 60/Finds Bag 3
1 :Turkey/Kenan Tepe/Area D/Trench 6/Locus 72/Finds Bag 2
1 :Turkey/Kenan Tepe/Area C/Trench 1/Locus 1039/Finds Bag 1305
1 :Turkey/Kenan Tepe/Area D/Trench 6/Locus 4/Finds Bag 45
1 :Turkey/Kenan Tepe/Area C/Trench 1/Locus 1109/Finds Bag 15
1 :Turkey/Kenan Tepe/Area D/Trench 6/Locus 4/Finds Bag 58
1 :Turkey/Kenan Tepe/Area D/Trench 4/Locus 40

                                790375
pottery                           9302
Terracotta                        3427
glass (material)                  2374
Bucchero                          1321
ceramic                           1249
ceramic (material)                1214
bronze                            1188
clay                              1154
iron (metal)                       833
sheet glass                        790
bone                               702
iron                               647
bone (material)                    633
glass                              523
ivory                              396
stone                              391
chert                              363
kaolin                             286
shell (animal material)            248
copper alloy                       196
antler                             181
plaster                            159
metal                              149
porcelain (material)               133
wood (plant material)    

                                                         793751
vessel                                                     2597
bottles                                                    2060
amphora                                                    2012
cooking-vessel                                             1864
plaque                                                     1097
sample                                                      957
spool                                                       894
panes (architectural elements)                              807
sima                                                        744
akroterion                                                  731
tile                                                        708
remains                                                     632
spindle-whorl                                               597
roof-tile                                                   567
tack                                    

Animal Bone                           444453
Object                                196547
Sample                                 76975
Plant remains                          34749
Architectural Element                  23215
Pottery                                22035
Sample, Collection, or Aggregation      7128
Shell                                   5554
Human Bone                              3736
Non Diagnostic Bone                     1332
Glass                                   1236
Bulk Ceramic                            1160
Coin                                    1060
Human Subject                            520
Groundstone                              272
Biological subject, Ecofact              232
Sculpture                                 50
Reference Collection                      18
Name: item category, dtype: int64

In [6]:
src="GEOME"
data = pd.read_csv('data/'+src+'.txt',sep='#', keep_default_na=False, encoding='utf-8')



C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10,25,26,41,49,89,149,175) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process

# data.columns
# #data[fuzz.ratio(data['parent_habitat'], 'arctostaphylos pungens')> 90]

# #arctostaphylos pungens

In [8]:
e = data['parent_habitat'].value_counts()
e
saveCounts2CSV(e, "GEOME habitat", 10)

f = data['parent_microHabitat'].value_counts()
f
saveCounts2CSV(e, "GEOME mircoHabitat", 10)


                                                                                                                                                                                                                                                                  135218
missing data                                                                                                                                                                                                                                                       10772
aquatic                                                                                                                                                                                                                                                            10611
intertidal zone                                                                                                                                                                                              

                                                                                                              194656
dead coral head                                                                                                 4719
ARMS                                                                                                            4554
Coastal reef, pocilopora dominated, clear visibility, good coral cover, 10 m                                    1015
in rubble                                                                                                        491
Drop off reef, porites dominated, clear visibility, 7m                                                           469
under rocks                                                                                                      416
water column                                                                                                     383
brushed from under rubble                                       

In [9]:
def getData(src):
    id_property = srcids[src]
    intrinsic_properties = srcs[src].copy()
    print(src)
    #print(id_property) 
    #print(intrinsic_properties)
    data = pd.DataFrame()
    data = pd.read_csv('data/'+src+'.txt',sep='#', keep_default_na=False, encoding='utf-8')
    #print(data)
    if not id_property:
        data['id'] = range(0,data.shape[0]) #when no id field is supplied
        id_property = 'id'
    
    #print(intrinsic_properties)
    #print(id_property)
    intrinsic_properties.append(id_property)
    #print(intrinsic_properties)
    data = data.filter(intrinsic_properties, axis=1)
    #print(data.columns) 
    data = data.rename(columns={id_property:'id'})
    data['src'] = src
    #data['id'] = src+':'+data['id'].astype(str)
    data['original']= ''

    for p in intrinsic_properties:
        if not p == id_property:
            #print('property: '+p)
            data['original'] = data[p].astype(str)+';'+data['original'] 
    #print(data)
    data['original'] = data['original'].str.replace(r";+", ";")
    data = data.dropna(subset = ['original']).reset_index(drop=True)
    data = data[['src', 'id', 'original']] #id + original
    data.reset_index(inplace=True, drop=True)              
    return data
                                
#test = getData("GEOME")
#print(test)
                    

In [10]:
def gatherTerm(cframe, threshold):
    tfidf_vectorizer = TfidfVectorizer(max_df = 1, max_features=500, min_df=0, stop_words='english',
                                  use_idf=True, ngram_range=(1, 3))
    try:
        tfidf_matrix = tfidf_vectorizer.fit_transform(cframe['description'])
        #print('tfidf_matrix')
        #print(tfidf_matrix[0,2])
        terms = tfidf_vectorizer.get_feature_names()
        #print('terms')
        #print(terms)
        ordered_index = tfidf_matrix.toarray().argsort()[:, ::-1]
        #print('ordered_index')
        #print(ordered_index) 

        #print("threshold:"+str(threshold))
        #print("length of ordered index:"+str(len(ordered_index)))
        #print("# of clusters:"+str(num_clusters))
        allterms = []
        for i in range(0, len(cframe)):
            #print('i='+str(i))
            n_terms = []
            for ind in ordered_index[i,]: 
                #print('ind='+str(ind))
                #print(ordered_index[i,])
                if tfidf_matrix[i, ind] > threshold:
                    n_terms.append(terms[ind]) 
            allterms.append(n_terms)
        return allterms
        
    except ValueError:#no term obtained from tfidf when all records holds the same set of terms
        return []

In [11]:
df_content_src = pd.DataFrame()

data_list=[]
data_list = Parallel(n_jobs=min(len(srcnames), cores), verbose=50)(delayed(getData)(src) for src in srcnames)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
GEOME


C:\ProgramData\Anaconda3\lib\site-packages\joblib\parallel.py:262: DtypeWarning: Columns (10,25,26,41,49,89,149,175) have mixed types.Specify dtype option on import or set low_memory=False.
  return [func(*args, **kwargs)
C:\Users\updates\AppData\Local\Temp/ipykernel_24924/2563802782.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  data['original'] = data['original'].str.replace(r";+", ";")


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.9s remaining:    0.0s
SMITHSONIAN
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    7.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    7.6s finished


In [12]:
df_content_src = pd.concat(data_list, ignore_index=True)
df_content_src['original_beforeclean'] = df_content_src['original']
df_content_src.shape #5,833,656
del data_list

(422974, 4)

In [13]:
nltk.download('stopwords')
estopwords = stopwords.words('english')+["sample", "samples", "sampling", "sampled", "sample_id", "cm"]
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\updates\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
def clean(line):
    if pd.isna(line):
        return ''
    else:
        newline =''
        for token1 in re.split(r'#+', line):
            for token2 in token1.split():
                token2 = token2.lower()
                if token2.startswith('https://') or token2.startswith('http://') or re.match(".*?\d.*", token2) or len(token2)<=2 or token2 in estopwords:
                    next
                else:
                    token2 = re.sub(r'['+string.punctuation+']', ' ', token2)
                    for token3 in re.split(r'[/<>. ]', token2):
                        if len(token3)<=2 or token3 in estopwords:
                            next
                        else:
                            newline = newline+' '+token3.strip()
                        
        return newline.strip()
    
    
# line = "Char WS-9513.01.06-b*""#event.###[blank]/records:/[blank]  [ *found * {}: [and at https://wwww.applies.com/###12 cm. l.d. ###width 1dm ###rock>mineral>blue mineral###"    
# # line = '""#""#""#""#"74"#""#""#"74"#""#""#""#""#""#""#""#""#"2020"#""#""#"University of Florida"#""#"23.8361515"#""#""#""#""#""#""#"Alpheidae sp. 1"#""#""#""#""#""#"BOMAN_3342"#""#""#""#""#""#"Oman"#""#"NSF_OMAN"#""#""#""#""#""#""#""#""#"OMAN_020A"#""#""#""#"NSF_OMAN"#""#""#""#""#""#""#"1"#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#"57.9767976"#""#"Event"#""#""#""#""#"Sample"#""#""#""#""#""#""#"in Pocillopora and Acropora rubble"#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#"Damanyat Islands, S of June Island"#""#""#""#""#""#""#""#""#""#""#""#""#"Abby Uehling"#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#"74"#""#"Abby Uehling"#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#"Muscat Governorate"#""#""#"OMAN_020A"#""#"Arthropoda"#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""#""'
# # re.split(r'#+', line)
# newline = clean(line)
# newline

In [15]:
allplines_src = list()
allplines_src = Parallel(n_jobs=cores)(delayed(clean)(line) for line in df_content_src['original'])


### frequency counts of combined fields from one or more collections

In [16]:
# all fields selected are combined into 'original'
df_content_src['original'] = allplines_src


In [17]:
for text in df_content_src['original']: 
    if 'arctostaphylos pungens' in text:
        temp = df_content_src[df_content_src['original'] == text]['original_beforeclean']
        for t in temp:
            print(t)
        



with Quercus arizonica, Q. edwardii, Pinus engelmannii, P. teocote, Ceanothus, Juniperus deppeana, Arctostaphylos pungens.;
with Quercus arizonica, Q. edwardii, Pinus engelmannii, P. teocote, Ceanothus, Juniperus deppeana, Arctostaphylos pungens.;
with Quercus arizonica, Q. edwardii, Pinus engelmannii, P. teocote, Ceanothus, Juniperus deppeana, Arctostaphylos pungens.;
with Quercus arizonica, Q. edwardii, Pinus engelmannii, P. teocote, Ceanothus, Juniperus deppeana, Arctostaphylos pungens.;
with Quercus arizonica, Q. edwardii, Pinus engelmannii, P. teocote, Ceanothus, Juniperus deppeana, Arctostaphylos pungens.;
with Quercus arizonica, Q. edwardii, Pinus engelmannii, P. teocote, Ceanothus, Juniperus deppeana, Arctostaphylos pungens.;
with Pinus engelmannii, P. teocote, Tagetes, Pycreus, Rhynchospora, Arctostaphylos pungens, Quercus arizonica, Q. edwardii,;
with Pinus engelmannii, P. teocote, Tagetes, Pycreus, Rhynchospora, Arctostaphylos pungens, Quercus arizonica, Q. edwardii,;
with P

Open slopes derived from igneous rocks with Muhlenbergia, Salvia, and Arctostaphylos pungens.;
Open slopes derived from igneous rocks with Muhlenbergia, Salvia, and Arctostaphylos pungens.;
Open slopes derived from igneous rocks with Muhlenbergia, Salvia, and Arctostaphylos pungens.;
Open slopes derived from igneous rocks with Muhlenbergia, Salvia, and Arctostaphylos pungens.;
Open slopes derived from igneous rocks with Muhlenbergia, Salvia, and Arctostaphylos pungens.;
Open slopes derived from igneous rocks with Muhlenbergia, Salvia, and Arctostaphylos pungens.;
Open slopes derived from igneous rocks with Muhlenbergia, Salvia, and Arctostaphylos pungens.;
Open slopes derived from igneous rocks with Muhlenbergia, Salvia, and Arctostaphylos pungens.;
Open slopes derived from igneous rocks with Muhlenbergia, Salvia, and Arctostaphylos pungens.;
Open slopes derived from igneous rocks with Muhlenbergia, Salvia, and Arctostaphylos pungens.;
Open slopes derived from igneous rocks with Muhlen

Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergia and Aristida.;
Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergia and Aristida.;
Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergia and Aristida.;
Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergia and Aristida.;
Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergia and Aristida.;
Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergia and Aristida.;
Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergia and Aristida.;
Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergia and Aristida.;
Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergi

Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergia and Aristida.;
Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergia and Aristida.;
Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergia and Aristida.;
Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergia and Aristida.;
Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergia and Aristida.;
Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergia and Aristida.;
Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergia and Aristida.;
Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergia and Aristida.;
Dry, rocky slopes with Quercus, Pinus cembroides, Arctostaphylos pungens, Juniperus, Muhlenbergi

Rocky slopes an along ridge with Quercus, Pinus, Arctostaphylos pungens, and Pinus cembroides;
Rocky slopes an along ridge with Quercus, Pinus, Arctostaphylos pungens, and Pinus cembroides;
Rocky slopes an along ridge with Quercus, Pinus, Arctostaphylos pungens, and Pinus cembroides;
Rocky slopes an along ridge with Quercus, Pinus, Arctostaphylos pungens, and Pinus cembroides;
Rocky slopes an along ridge with Quercus, Pinus, Arctostaphylos pungens, and Pinus cembroides;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenb

...

Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes

Rocky slopes an along ridge with Quercus, Pinus, Arctostaphylos pungens, and Pinus cembroides;
Rocky slopes an along ridge with Quercus, Pinus, Arctostaphylos pungens, and Pinus cembroides;
Rocky slopes an along ridge with Quercus, Pinus, Arctostaphylos pungens, and Pinus cembroides;
Rocky slopes an along ridge with Quercus, Pinus, Arctostaphylos pungens, and Pinus cembroides;
Rocky slopes an along ridge with Quercus, Pinus, Arctostaphylos pungens, and Pinus cembroides;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenb

Rocky slopes with Quercus, Arctostaphylos pungens, and Muhlenbergia.;
Rocky slopes with Quercus, Arctostaphylos pungens, and Muhlenbergia.;
Rocky slopes with Quercus, Arctostaphylos pungens, and Muhlenbergia.;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Querc

Along ridge and exposed bedrock with Quercus, Arctostaphylos pungens, Microchloa, and Muhlenbergia.;
Along ridge and exposed bedrock with Quercus, Arctostaphylos pungens, Microchloa, and Muhlenbergia.;
Along ridge and exposed bedrock with Quercus, Arctostaphylos pungens, Microchloa, and Muhlenbergia.;
Along ridge and exposed bedrock with Quercus, Arctostaphylos pungens, Microchloa, and Muhlenbergia.;
Along ridge and exposed bedrock with Quercus, Arctostaphylos pungens, Microchloa, and Muhlenbergia.;
Along ridge and exposed bedrock with Quercus, Arctostaphylos pungens, Microchloa, and Muhlenbergia.;
Along ridge and exposed bedrock with Quercus, Arctostaphylos pungens, Microchloa, and Muhlenbergia.;
Along ridge and exposed bedrock with Quercus, Arctostaphylos pungens, Microchloa, and Muhlenbergia.;
Along ridge and exposed bedrock with Quercus, Arctostaphylos pungens, Microchloa, and Muhlenbergia.;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes wi

Steep slopes below ridge with Pinus, Quercus, Vaccinium, Arctostaphylos pungens, Thalictrum, Juniperus, and Muhlenbergia;
Steep slopes below ridge with Pinus, Quercus, Vaccinium, Arctostaphylos pungens, Thalictrum, Juniperus, and Muhlenbergia;
Steep slopes below ridge with Pinus, Quercus, Vaccinium, Arctostaphylos pungens, Thalictrum, Juniperus, and Muhlenbergia;
Steep slopes below ridge with Pinus, Quercus, Vaccinium, Arctostaphylos pungens, Thalictrum, Juniperus, and Muhlenbergia;
Calcareous rocky slopes with Quercus greggii, Arctostaphylos pungens, Symphoricarpos macrophyllus, Arbutus bicolor, and scattered Pinus spp.;
Calcareous rocky slopes with Quercus greggii, Arctostaphylos pungens, Symphoricarpos macrophyllus, Arbutus bicolor, and scattered Pinus spp.;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;
Rocky slopes with Pinus, Quercus, Arctostaphylos pungens, and Muhlenbergia;


In [18]:
g = df_content_src['original'].value_counts()
g
saveCounts2CSV(g, "three habitats", 10)


                                                                                                                 293804
missing data                                                                                                      10772
aquatic                                                                                                           10611
coral reef                                                                                                         6110
intertidal zone                                                                                                    4359
                                                                                                                  ...  
shady disturbed temperate forest glen igneous alnus sambucus acer ferns mentha longifolia microstegium common         1
dry rocky mountain slopes sparse vegetation                                                                           1
psammocora obtusangula                  

In [19]:

print("df_content_src.shape[0] before dropna:"+str(df_content_src.shape[0])) 
df_content_src = df_content_src.replace(r'^\s*$', np.NaN, regex=True)
df_content_src = df_content_src.dropna(how='any').reset_index(drop=True)
print("df_content_src.shape[0] after dropna:"+str(df_content_src.shape[0])) 
df_content_src.drop_duplicates(subset='original', ignore_index=True, inplace=True) #remove redundant rows
print("df_content_src.shape[0] after deduplicationa:"+str(df_content_src.shape[0])) 
pd.unique(df_content_src['src'])
plines_src = df_content_src['original']

#del allplines_src

df_content_src.shape[0] before dropna:422974
df_content_src.shape[0] after dropna:129170
df_content_src.shape[0] after deduplicationa:6151


array(['GEOME', 'SMITHSONIAN'], dtype=object)

In [20]:
model = fasttext.load_model('data/cc.en.300.bin') #takes 7GB

In [21]:
vlines_src = list() #records represented as vectors
for pline in plines_src:
    vlines_src.append(model.get_sentence_vector(pline))

#normalize to unit vector
vlines_src = normalize(vlines_src, axis=1)
print("len(vlines_src):")
print(len(vlines_src))
#del model #reclaim memory
#print(vlines_src[0])
#print(len(vlines_src)) 


#with open("vlines_src.all.fasttext"+".pkl", 'wb') as outp:
#    pickle.dump(vlines_src, outp, pickle.HIGHEST_PROTOCOL)

len(vlines_src):
6151


In [22]:
def birch(rep, vlines_src):
    brc_src = Birch(branching_factor=20000, n_clusters=12, threshold=0.5) #n_clusters = None, meaning unlimited  
    brc_src.fit(vlines_src)

    labels_src = brc_src.predict(vlines_src)
    df_src = pd.DataFrame({'src':df_content_src['src'], 'id':df_content_src['id'], 'birchcluster':labels_src, 'content':plines_src, 'original':df_content_src['original']}) 

    tab = df_src.groupby(['src','birchcluster']).size()
    print(rep+" source clusters:")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        #print(tab.unstack().transpose())
        print(tab.transpose())

    #with open("clusters.src.birch."+rep+".pkl", 'wb') as outp:
     #   pickle.dump(df_src, outp, pickle.HIGHEST_PROTOCOL)

  
    return df_src


In [23]:
rep = 'fasttext'
df = birch(rep, vlines_src)

fasttext source clusters:
src          birchcluster
GEOME        0               859
             1               111
             2                47
             3                74
             4                29
             5               193
             6                83
             7               180
             8               248
             9                31
             10               82
             11               35
SMITHSONIAN  0               186
             1               831
             2               493
             3               396
             4               627
             5               691
             6                31
             7                52
             8               222
             9                51
             10               92
             11              507
dtype: int64


In [24]:
# with open("clusters.src.birch."+rep+".pkl", 'rb') as inp:
#     df = pickle.load(inp)
    
# df

In [25]:
#produce terms from Birch clustering result

frame = df[['birchcluster','content']]
frame = frame.rename(columns={'birchcluster':'cluster', 'content':'description'})
cframe =frame.groupby('cluster', as_index = False).agg({'description': ' '.join})
terms = gatherTerm(cframe, 0.1)
i=0
for term in terms:
    print('cluster '+ str(cframe.iloc[i]['cluster'])+':')
    print(term)
    i = i+1
    print()

cluster 0:
['intertidal zone', 'marine', 'silty fringing', 'silty fringing reef', 'patch reef', 'reef biome', 'subtidal', 'fringing reef bay', 'reef bay', 'barrier reef', 'rocky reef', 'coral reef', 'estuarine', 'estuarine biome', 'rubble outer', 'subtidal rocky reef', 'subtidal rocky', 'marine subtidal rocky', 'marine subtidal', 'rocky reef biome', 'rubble outer reef', 'reef wall']

cluster 1:
['second', 'second growth', 'road ecological', 'road ecological remarks', 'diameter', 'sun', 'collector yes trail', 'yes trail', 'river ecological', 'river ecological remarks', 'blackwater', 'blackwater river', 'ambre', 'collector yes tall', 'montagne ambre', 'creek ecological', 'creek ecological remarks', 'montagne', 'dirt road', 'collector yes ground', 'yes tall', 'yes ground']

cluster 2:
['yucca', 'pungens', 'virginiana', 'divaricata', 'arctostaphylos pungens', 'larrea divaricata', 'canadensis', 'rubra', 'piptochaetium', 'dolomite glade', 'pinus cembroides', 'cembroides', 'muhlenbergii', 'ar

In [26]:
# ## clustering of subclusters of BIRCH result with parallal programming

from fastcluster import linkage_vector
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import dendrogram


#birchcluster_df: the input observation data (D=300xN)
#birchcluster_number:  cluster number of birchcluster_df in BIRCH result 
#min_obs: minimal observations in birchcluster_df to run hierarichal clustering, must be >=2

def h_clustering_fast(birchcluster_df, birchcluster_number, link_method, min_obs=2 ):
    if len(birchcluster_df) < min_obs:
        return [] 
    else:
        linkage_matrix = fastcluster.linkage_vector(birchcluster_df, link_method) #single, complete, average, weighted, median, centroid, ward
        return linkage_matrix
    
#result = h_clustering_fast(vlines[df.index[df['birchcluster']==0]], 0, 'ward', 20)
#result
    





#birchcluster_df: df holding the birchcluster observation content
#birchcluster_number:  cluster number of birchcluster_df in BIRCH result 
#linkage_matrix: the linkage_matrix for the birchcluster
#t: threshold to obtain clusters from hierarchical clustering for the specified criterion
#criterion: criterion used to obtain clusters

def obtain_h_clusters(birchcluster_df, birchcluster_number, linkage_matrix, t=10, criterion="maxclust"):
    if np.any(linkage_matrix):
        clusters = fcluster(linkage_matrix, t, criterion=criterion) #distance, inconsistency
        num_clusters = len(np.unique(clusters))
        birchcluster_df = birchcluster_df.reset_index()
        cresult = birchcluster_df[['index']]
        cresult['hcluster']= clusters
         
        #index lines with clusters
        records = {'description':df[df['birchcluster']==birchcluster_number].content, 'cluster':clusters, 'birchcluster':birchcluster_number}
        frame = pd.DataFrame(records, columns=['description', 'cluster', 'birchcluster'])
        cframe =frame.groupby('cluster', as_index = True).agg({'description': ' '.join})
        return cresult
    else:
        return []
    

    
#len(results)
#16 empty
#cresult = obtain_h_clusters(df[df['birchcluster']==3], 3, linkage_matrix=results[3], t=3 if df[df['birchcluster']==3].shape[0] < 100 else 10 , criterion="maxclust")
#cresult
    



In [27]:
results = []        
results = Parallel(n_jobs=8, verbose=1)(delayed(h_clustering_fast)(vlines_src[df.index[df['birchcluster']==c]], c, 'ward', 2)  for c in range(0, len(df.birchcluster.unique())))
print("# of linkage matrices obtained: "+str(len(results)))

cresultlist = []
for cresult in Parallel(n_jobs=8,verbose=1)(delayed(obtain_h_clusters)(df[df['birchcluster']==c], c, linkage_matrix=results[c], t=3 if df[df['birchcluster']==c].shape[0] < 100 else 10, criterion='maxclust') for c in range(0, len(df.birchcluster.unique()))):
    cresultlist.append(cresult)


combined = pd.DataFrame()
#concat list of cresult row-wise
for cresult in cresultlist:
    combined = pd.concat([combined, cresult], ignore_index=True)
        

#print(combined)    
combined.set_index('index', inplace=True)

#df now holds all the results
df = df.join(combined) 




[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  10 out of  12 | elapsed:    1.7s remaining:    0.3s
[Parallel(n_jobs=8)]: Done  12 out of  12 | elapsed:    1.7s finished


# of linkage matrices obtained: 12


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  10 out of  12 | elapsed:    0.9s remaining:    0.1s
[Parallel(n_jobs=8)]: Done  12 out of  12 | elapsed:    0.9s finished


In [28]:

#produce terms from Birch + hierarchical clustering results

df["cluster"] = df["birchcluster"].astype(str) +'-'+ df["hcluster"].astype(str)
frame = df[['cluster','content']]
frame = frame.rename(columns={'content':'description'})
cframe =frame.groupby('cluster', as_index = False).agg({'description': ' '.join})
terms = gatherTerm(cframe, 0.1)
i=0
allterms = list()
for term in terms:
    print('cluster '+cframe.iloc[i]['cluster']+':')
    print(term)
    i = i+1
    print()
    allterms.extend(term)






cluster 0-1:
['rubble outer', 'rubble outer reef', 'deep reef slope', 'near reef', 'silty patch reef', 'silty patch', 'reef outer', 'reef outer reef', 'brushed rubble outer', 'slope outer', 'near reef crest', 'reef slope rocks', 'rocks outer', 'outer barrier reef', 'outer barrier', 'wall coral', 'patch reef slope', 'wall coral silty', 'patch reef steep', 'reef slope outer', 'night reef', 'coral silty slope', 'crest reef']

cluster 0-10:
['chapman', 'surge channel', 'lagoon high current', 'biome boat', 'high current', 'chapman biome', 'strong', 'chapman biome boat', 'lagoon strong', 'lagoon strong current', 'crest narrow lagoon', 'treatment', 'treatment outflow', 'strong current']

cluster 0-2:
['sandflat coral', 'massive', 'sandflat coral bommies', 'massive corals', 'rocks sandflat']

cluster 0-3:
['flat reef', 'reef fringing', 'reef fringing reef', 'reef padina turbinaria', 'fringing reef padina', 'turbinaria sand hard', 'turbinaria sand', 'bits', 'shallow silty fringing', 'reef bits'

In [29]:
from collections import Counter
counter = Counter(allterms)
counter.most_common()

[('rubble outer', 1),
 ('rubble outer reef', 1),
 ('deep reef slope', 1),
 ('near reef', 1),
 ('silty patch reef', 1),
 ('silty patch', 1),
 ('reef outer', 1),
 ('reef outer reef', 1),
 ('brushed rubble outer', 1),
 ('slope outer', 1),
 ('near reef crest', 1),
 ('reef slope rocks', 1),
 ('rocks outer', 1),
 ('outer barrier reef', 1),
 ('outer barrier', 1),
 ('wall coral', 1),
 ('patch reef slope', 1),
 ('wall coral silty', 1),
 ('patch reef steep', 1),
 ('reef slope outer', 1),
 ('night reef', 1),
 ('coral silty slope', 1),
 ('crest reef', 1),
 ('chapman', 1),
 ('surge channel', 1),
 ('lagoon high current', 1),
 ('biome boat', 1),
 ('high current', 1),
 ('chapman biome', 1),
 ('strong', 1),
 ('chapman biome boat', 1),
 ('lagoon strong', 1),
 ('lagoon strong current', 1),
 ('crest narrow lagoon', 1),
 ('treatment', 1),
 ('treatment outflow', 1),
 ('strong current', 1),
 ('sandflat coral', 1),
 ('massive', 1),
 ('sandflat coral bommies', 1),
 ('massive corals', 1),
 ('rocks sandflat', 1)